In [2]:
import pandas as pd
import os

In [3]:
os.listdir("inside_airbnb") 

['.DS_Store',
 'calendar.csv',
 'listings.csv',
 'listings_summ.csv',
 'neighbourhoods.csv',
 'reviews.csv',
 'reviews_summ.csv']

In [4]:
listings = pd.read_csv('inside_airbnb/listings.csv')
list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
reviews = pd.read_csv('inside_airbnb/reviews.csv')
reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [5]:
listings.columns

Index([u'id', u'listing_url', u'scrape_id', u'last_scraped', u'name',
       u'summary', u'space', u'description', u'experiences_offered',
       u'neighborhood_overview', u'notes', u'transit', u'access',
       u'interaction', u'house_rules', u'thumbnail_url', u'medium_url',
       u'picture_url', u'xl_picture_url', u'host_id', u'host_url',
       u'host_name', u'host_since', u'host_location', u'host_about',
       u'host_response_time', u'host_response_rate', u'host_acceptance_rate',
       u'host_is_superhost', u'host_thumbnail_url', u'host_picture_url',
       u'host_neighbourhood', u'host_listings_count',
       u'host_total_listings_count', u'host_verifications',
       u'host_has_profile_pic', u'host_identity_verified', u'street',
       u'neighbourhood', u'neighbourhood_cleansed',
       u'neighbourhood_group_cleansed', u'city', u'state', u'zipcode',
       u'market', u'smart_location', u'country_code', u'country', u'latitude',
       u'longitude', u'is_location_exact', u'prope

In [6]:
listings = listings.loc[(listings.zipcode==92101) & (listings.room_type == 'Entire home/apt')]

In [7]:
calendar['price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [8]:
calendar['price'] = pd.to_numeric(calendar['price'], errors='coerce')

In [9]:
calendar['price'] = calendar['price']/100

In [10]:
def group_mean(x):
    x['mean_price'] = x['price'].mean()
    return x

In [11]:
fr = calendar.groupby(calendar.listing_id).apply(group_mean)

In [12]:
fr = fr.loc[fr.available!='t']

In [13]:
frame = fr[['listing_id','mean_price']].groupby(['listing_id']).sum()

In [14]:
merged = listings.merge(frame, how='inner', left_on='id', right_index=True)

In [15]:
merged.rename(columns={'mean_price':'estimated_tot_income'}, inplace=True)

In [16]:
merged['estimated_tot_income'] = merged['estimated_tot_income'].apply(lambda x: x*0.75)

In [17]:
cols = ['id','bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet','review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'estimated_tot_income']

In [18]:
merged[cols].sort_values(['estimated_tot_income'], ascending=False)

,id,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,estimated_tot_income
5731,6869868,3.0,2.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,""Air Conditioning"",Kit...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,599062.500000
1285,6275771,2.0,1.0,2.0,Real Bed,"{TV,Internet,""Wireless Internet"",""Air Conditio...",NaN,10.0,10.0,10.0,10.0,10.0,10.0,351000.000000
5723,13298618,3.0,3.0,5.0,Real Bed,"{TV,""Wireless Internet"",""Air Conditioning"",Kit...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,341978.048780
1174,13634611,1.0,1.0,1.0,Real Bed,"{TV,""Wireless Internet"",""Air Conditioning"",Kit...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,228225.000000
1046,13016622,2.0,2.0,2.0,Real Bed,"{TV,""Wireless Internet"",""Air Conditioning"",Poo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,218400.000000
894,13732088,2.0,3.0,3.0,Real Bed,"{TV,""Wireless Internet"",""Air Conditioning"",Poo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,202791.750000
1222,3675793,2.0,2.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",NaN,10.0,10.0,10.0,10.0,10.0,10.0,177450.000000
862,13657511,1.5,2.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,160850.812500
1385,3676708,2.0,2.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",NaN,10.0,9.0,10.0,10.0,10.0,9.0,154810.526316
993,2665030,2.0,2.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",NaN,10.0,10.0,10.0,10.0,10.0,10.0,149592.857143


In [19]:
calendar.loc[(calendar.listing_id == 6869868) & (calendar.available!='t')].count()

listing_id    355
date          355
available     355
price           0
dtype: int64